Bu ödevde, önceki derste Titanic verileri ile oluşturduğunuz modelin performansını ölçeceksiniz. Bu ödevi tamamlamak için, aşağıdaki görevlerin çözümlerini içeren bir Jupyter not defterini link olarak gönderin. Kafanıza takılan konuları mentörünüzle konuşabilir ya da ofis saatinde Slack üzerinden sorabilirsiniz.

    Modelinizin performansı çapraz doğrulama yaparak ve farklı metrikler kullanarak değerlendirin.
    Hiperparemetre ayarlaması yaparak en uygun parametrelere sahip modeli belirleyin.

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, PolynomialFeatures
import warnings; warnings.filterwarnings("ignore")


In [9]:
train_data = pd.read_csv("traintitanic.csv")
train_data.isnull().sum()*100/train_data.shape[0]

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [10]:
train_data["Age"].fillna(train_data["Age"].mean(), inplace=True)
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C


In [11]:
train_data = pd.get_dummies(train_data, columns = ['Sex','Embarked',"Pclass"],drop_first=True)

In [12]:
y = train_data["Survived"]
X = train_data.drop(columns = ["PassengerId","Survived","Name","Ticket","Cabin"])


In [13]:
log_reg = LogisticRegression()

In [16]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
x_egitim, x_test, y_egitim, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

parts = kf.split(X_train)
for num, (egitim_index, test_index) in enumerate(parts):
    print("{}.Eğitim Kümesi Boyutu : {}".format(num+1,len(egitim_index)))
    print("{}.Test Kümesi Boyutu   : {}".format(num+1,len(test_index)))
    print('-'*27)

1.Eğitim Kümesi Boyutu : 712
1.Test Kümesi Boyutu   : 179
---------------------------
2.Eğitim Kümesi Boyutu : 713
2.Test Kümesi Boyutu   : 178
---------------------------
3.Eğitim Kümesi Boyutu : 713
3.Test Kümesi Boyutu   : 178
---------------------------
4.Eğitim Kümesi Boyutu : 713
4.Test Kümesi Boyutu   : 178
---------------------------
5.Eğitim Kümesi Boyutu : 713
5.Test Kümesi Boyutu   : 178
---------------------------


In [17]:
score_list = []
pieces = kf.split(X_train)
## pieces'i her seferinde her hücreye yazmak gerekiyor. Tanımlanmıyor.
for i, (train_index, test_index) in enumerate(pieces):
    x_train, y_train = X_train[train_index], Y_train[train_index]
    x_test, y_test = X_train[test_index], Y_train[test_index]
    
    log_reg.fit(x_train, y_train)
    
    prediction = log_reg.predict(x_test)
    
    score = log_reg.score(x_test, y_test)  
    score_list.append(score)
    
    print("{}. parça skoru  : {:.3f}".format(i+1, score))
    print("-"*23, "\n")

1. parça skoru  : 0.810
----------------------- 

2. parça skoru  : 0.781
----------------------- 

3. parça skoru  : 0.854
----------------------- 

4. parça skoru  : 0.770
----------------------- 

5. parça skoru  : 0.787
----------------------- 



In [18]:
print("Ortalama doğruluk değeri : {:.2f}".format(np.mean(score_list)))


Ortalama doğruluk değeri : 0.80


In [20]:
from sklearn.model_selection import cross_validate, cross_val_score

cv = cross_validate(estimator=log_reg,
                     X=X_train,
                     y=Y_train,
                     cv=10,
                     return_train_score=True, 
                     scoring = ['accuracy', 'precision', 'r2']
                    )
print('Train Scores :', cv['train_accuracy'], sep = '\n')
print("-"*50)
print('Test Scores     :', cv['test_accuracy'], sep = '\n')

Train Scores :
[0.80898876 0.8042394  0.81047382 0.79177057 0.80049875 0.80299252
 0.80049875 0.80299252 0.80548628 0.79551122]
--------------------------------------------------
Test Scores     :
[0.78888889 0.79775281 0.7752809  0.80898876 0.78651685 0.7752809
 0.78651685 0.78651685 0.80898876 0.84269663]


In [21]:
print('Test Kümesi   Ortalaması : ', cv['test_accuracy'].mean())
print('Eğitim Kümesi Ortalaması : ', cv['train_accuracy'].mean())

Test Kümesi   Ortalaması :  0.7957428214731586
Eğitim Kümesi Ortalaması :  0.8023452604443946


In [22]:
print('Test Kümesi Doğruluk Ortalaması     : {:.2f}'.format(cv['test_accuracy'].mean()))
print('Test Kümesi R-kare  Ortalaması      : {:.2f}'.format(cv['test_r2'].mean()))
print('Test Kümesi Hassasiyet Ortalaması   : {:.2f}'.format(cv['test_precision'].mean()))
print('Eğitim Kümesi Doğruluk Ortalaması   : {:.2f}'.format(cv['train_accuracy'].mean()))
print('Eğitim Kümesi R-kare  Ortalaması    : {:.2f}'.format(cv['train_r2'].mean()))
print('Eğitim Kümesi Hassasiyet Ortalaması : {:.2f}'.format(cv['train_precision'].mean()))

Test Kümesi Doğruluk Ortalaması     : 0.80
Test Kümesi R-kare  Ortalaması      : 0.14
Test Kümesi Hassasiyet Ortalaması   : 0.75
Eğitim Kümesi Doğruluk Ortalaması   : 0.80
Eğitim Kümesi R-kare  Ortalaması    : 0.16
Eğitim Kümesi Hassasiyet Ortalaması : 0.76


In [23]:
cv = cross_val_score(estimator=log_reg,
                     X=X_train,
                     y=Y_train,
                     cv=10                    
                    )
print('Model Skorları            : ', cv, sep = '\n')

Model Skorları            : 
[0.78888889 0.79775281 0.7752809  0.80898876 0.78651685 0.7752809
 0.78651685 0.78651685 0.80898876 0.84269663]


In [24]:
# grid search
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression()
parameters = {"C": [10 ** x for x in range (-5, 5, 1)],
              "penalty": ['l1', 'l2'], 
              "solver":["saga"]
             }

grid_cv = GridSearchCV(estimator=log_reg,
                       param_grid = parameters,
                       cv = 10)
grid_cv.fit(X_train, Y_train)
print("En iyi parametreler : ", grid_cv.best_params_)
print("En iyi skor         : ", grid_cv.best_score_)

En iyi parametreler :  {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}
En iyi skor         :  0.8013732833957553


In [25]:
results = grid_cv.cv_results_
df = pd.DataFrame(results)
df = df[['param_penalty','param_C', 'mean_test_score']]
df = df.sort_values(by='mean_test_score', ascending = False)
df

,param_penalty,param_C,mean_test_score
7,l2,0.01,0.801373
9,l2,0.1,0.795755
10,l1,1,0.795743
11,l2,1,0.795743
18,l1,10000,0.795743
17,l2,1000,0.795743
16,l1,1000,0.795743
15,l2,100,0.795743
14,l1,100,0.795743
13,l2,10,0.795743


In [26]:
from sklearn.model_selection import RandomizedSearchCV
rs_cv = RandomizedSearchCV(estimator=log_reg,
                           param_distributions = parameters,
                           cv = 10,
                           n_iter = 10,
                           random_state = 42,
                           scoring = "accuracy"
                      )
rs_cv.fit(X_train, Y_train)

print("En iyi parametreler        : ", rs_cv.best_params_)
print("En iyi hassasiyet değeri   : ", rs_cv.best_score_)

En iyi parametreler        :  {'solver': 'saga', 'penalty': 'l2', 'C': 1000}
En iyi hassasiyet değeri   :  0.7957428214731586


In [27]:
results = rs_cv.cv_results_
df = pd.DataFrame(results)
df = df[['param_penalty','param_C', 'mean_test_score']]
df = df.sort_values(by='mean_test_score', ascending = False)
df

,param_penalty,param_C,mean_test_score
1,l2,1000,0.795743
2,l2,100,0.795743
6,l2,1,0.795743
8,l1,10000,0.795743
9,l1,1000,0.795743
4,l1,0.1,0.791248
5,l2,0.001,0.673421
0,l1,0.00001,0.616167
3,l2,0.00001,0.616167
7,l2,0.0001,0.616167
